In [1]:
import pandas as pd 
import numpy as np
from datetime import datetime
from tqdm import tqdm
import pickle

In [2]:
import config
from scripts.utils import setup_web3, get_logs, save_raw_logs, load_csv_data
from scripts.fetch_logs import *
from scripts.process_validator_delegator import *
from scripts.process_user_rewards_vault import *

/Users/benoitrobaglia/miniforge3/envs/beraboost/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
w3 = setup_web3()
last_block = w3.eth.get_block("latest")["number"]
print(last_block)

2457270


In [5]:
timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

In [6]:
fetch_validator_delegator_logs(w3, 1, last_block, timestamp)

Fetching validator-delegator logs from block 1 to 2457270...
Found 40023 validator-delegator logs
Saved 40023 raw logs to raw_logs/validator_delegator/logs_2025-03-17_11:26:58.pkl


In [7]:
file_name = 'logs_2025-03-17_11:26:58.pkl'

In [8]:
raw_logs = pickle.load(open(f'raw_logs/validator_delegator/{file_name}', 'rb'))

In [9]:
decoded_logs = decode_all_validator_delegator_logs_multiprocessing(raw_logs)

Decoding 40023 logs using 11 processes...


Decoding logs: 100%|██████████| 40023/40023 [00:02<00:00, 15926.29it/s]


In [10]:
validator_delegator_state = decoded_logs.groupby(['validator_address', 'delegator_address']).amount_bgt_delegated.sum().reset_index()

In [11]:
c1_pubkey = '0xb6a44182b44298daee8e3773048b978dc36b19c71a47f90318ea5d5309fc06e9'
validator_delegator_state.loc[validator_delegator_state.validator_address == c1_pubkey].sort_values('amount_bgt_delegated', ascending=False)

,validator_address,delegator_address,amount_bgt_delegated
3754,0xb6a44182b44298daee8e3773048b978dc36b19c71a47...,0xf2048c29ef806ed003dee1ae703f00ef7340ac84,2.180500e+03
3749,0xb6a44182b44298daee8e3773048b978dc36b19c71a47...,0xca11bde05977b3631167028862be2a173976ca11,1.840340e+03
3753,0xb6a44182b44298daee8e3773048b978dc36b19c71a47...,0xef34e52ee7f94b10a64d8b23dc69c1aef92306d2,1.000000e+02
3743,0xb6a44182b44298daee8e3773048b978dc36b19c71a47...,0x7fb31cd942113428c528a503e2fdd8bb2ae91b7b,4.472917e+01
3734,0xb6a44182b44298daee8e3773048b978dc36b19c71a47...,0x0b9b90bcf3fa83a933668e072611af7805fe84ad,3.844242e+01
3750,0xb6a44182b44298daee8e3773048b978dc36b19c71a47...,0xe1cd1883cd6d05f971cfc148e2e7bc9802ff4849,3.573049e+01
3740,0xb6a44182b44298daee8e3773048b978dc36b19c71a47...,0x5ffcf42e97c07b2b418b42365752cd949ee2e2b7,1.579628e+01
3745,0xb6a44182b44298daee8e3773048b978dc36b19c71a47...,0x8b4e7ddcc28d7eba887d03b93f600d4030d4d586,4.247414e+00
3751,0xb6a44182b44298daee8e3773048b978dc36b19c71a47...,0xe427cfa8de00a6f688251350af71212c748a33d8,2.000000e+00
3738,0xb6a44182b44298daee8e3773048b978dc36b19c71a47...,0x5171f2d223b5f807456fdc12b10128addd187cd2,1.000000e+00
